In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 5
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = 128  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    
class Config(object):
    sequenceLength = 180  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 180)
train label shape: (20000, 1)
eval data shape: (5000, 180)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 模型构建

class Transformer(object):
    """
    Transformer Encoder 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        ma = self.multiheadAttention(queries=self.embeddedWords, keys=self.embeddedWords)
        outputs = self.feedForward(ma, [config.model.hiddenSizes, config.model.embeddingSize])
        outputs = tf.reshape(outputs, [-1, config.sequenceLength * config.model.embeddingSize])
        
        outputSize = outputs.get_shape()[-1].value
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(outputs, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
    def layerNormalization(self, inputs, epsilon=1e-8, scope="ln", reuse=None):
        # 本质上可以看作是BN层
        
        with tf.variable_scope(scope, reuse=reuse):
            inputsShape = inputs.get_shape() # [batch_size, time_step, hidden_size]
            print(inputsShape)
            paramsShape = inputsShape[-1:]
            print(paramsShape)
            # 在最后的维度上计算输入的数据的均值和方差
            mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
            beta = tf.Variable(tf.zeros(paramsShape))
            gamma = tf.Variable(tf.ones(paramsShape))
            normalized = (inputs - mean) / ((variance + epsilon) ** .5)
            outputs = gamma * normalized + beta

        return outputs
            
    def multiheadAttention(self, queries, keys, numUnits=None, numHeads=8, causality=False, scope="multihead_attention", reuse=None):
        
        with tf.variable_scope(scope, reuse=reuse):
            if numUnits is None:  # 若是没传入值，直接去输入数据的最后一维，即embedding size.
                numUnits = queries.get_shape().as_list()[-1]

            # tf.layers.dense可以做多维tensor数据的非线性映射，在计算self-Attention时，一定要对这三个值进行非线性映射，
            # 其实这一步就是论文中Multi-Head Attention中的对分割后的数据进行权重映射的步骤，我们在这里先映射后分割，原则上是一样的。
            # Q, K, V的维度都是[batch_size, time_step, hidden_size]
            Q = tf.layers.dense(queries, numUnits, activation=tf.nn.relu)
            K = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
            V = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
            
            # 将数据按最后一维分割成num_heads个, 然后按照第一维拼接
            # Q, K, V 的维度都是[batch_size * num_heads, time_step, hidden_size/num_heads]
            Q_ = tf.concat(tf.split(Q, numHeads, axis=-1), axis=0) 
            K_ = tf.concat(tf.split(K, numHeads, axis=-1), axis=0) 
            V_ = tf.concat(tf.split(V, numHeads, axis=-1), axis=0)
            
            # 计算keys和queries之间的点积，维度[batch_size * num_heads, queries_len, key_len], 后两维是queries和keys的序列长度
            similary = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))
            
            # 对计算的点积进行缩放处理，除以向量长度的根号值
            scaledSimilary = similary / (K_.get_shape().as_list()[-1] ** 0.5)
            
            # 在我们输入的序列中会存在padding这个样的填充词，这种词应该对最终的结果是毫无帮助的，原则上说当padding都是输入0时，
            # 计算出来的权重应该也是0，但是在transformer中引入了位置向量，当和位置向量相加之后，其值就不为0了，因此在添加位置向量
            # 之前，我们需要将其mask为0。虽然在queries中也存在这样的填充词，但原则上模型的结果之和输入有关，而且在self-Attention中
            # queryies = keys，因此只要一方为0，计算出的权重就为0。
            # 具体关于key mask的介绍可以看看这里： https://github.com/Kyubyong/transformer/issues/3
            
            # 将每一时序上的向量中的值相加取平均值
            keyMasks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1)))  # 维度[batch_size, time_step]
            
            # 利用tf，tile进行张量扩张， 维度[batch_size * num_heads, keys_len] keys_len = keys 的序列长度
            keyMasks = tf.tile(keyMasks, [numHeads, 1]) 
            
            # 增加一个维度，并进行扩张，得到维度[batch_size * num_heads, queries_len, keys_len]
            keyMasks = tf.tile(tf.expand_dims(keyMasks, 1), [1, tf.shape(queries)[1], 1])
            
            # tf.ones_like生成元素全为1，维度和scaledSimilary相同的tensor, 然后得到负无穷大的值
            paddings = tf.ones_like(scaledSimilary) * (-2 ** (32 + 1))
            
            # tf.where(condition, x, y),condition中的元素为bool值，其中对应的True用x中的元素替换，对应的False用y中的元素替换
            # 因此condition,x,y的维度是一样的。下面就是keyMasks中的值为0就用paddings中的值替换
            maskedSimilary = tf.where(tf.equal(keyMasks, 0), paddings, scaledSimilary) # 维度[batch_size * num_heads, queries_len, key_len]
            
            # 在计算当前的词时，只考虑上文，不考虑下文，出现在Transformer Decoder中。在文本分类时，可以只用Transformer Encoder。
            # Decoder是生成模型，主要用在语言生成中
            if causality:
                diagVals = tf.ones_like(maskedSimilary[0, :, :])  # [queries_len, keys_len]
                tril = tf.contrib.linalg.LinearOperatorTriL(diagVals).to_dense()  # [queries_len, keys_len]
                masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(maskedSimilary)[0], 1, 1])  # [batch_size * num_heads, queries_len, keys_len]

                paddings = tf.ones_like(masks) * (-2 ** (32 + 1))
                maskedSimilary = tf.where(tf.equal(masks, 0), paddings, maskedSimilary)  # [batch_size * num_heads, queries_len, keys_len]

            # 通过softmax计算权重系数，维度 [batch_size * num_heads, queries_len, keys_len]
            weights = tf.nn.softmax(maskedSimilary)

            # 加权和得到输出值, 维度[batch_size * num_heads, time_step, hidden_size/num_heads]
            outputs = tf.matmul(weights, V_)
            print(outputs)
            
            # 将多头Attention计算的得到的输出重组成最初的维度[batch_size, time_step, hidden_size]
            outputs = tf.concat(tf.split(outputs, numHeads, axis=0), axis=2)

            # 建立残差连接
            outputs += queries
            print(outputs)
            # normalization 层
            outputs = self.layerNormalization(outputs)
            return outputs

    def feedForward(self, inputs, numUnits=[2048, 512], scope="multiheadAttention", reuse=None):
        # 在这里的前向传播采用卷积神经网络
        with tf.variable_scope(scope, reuse=reuse):
            # 内层
            params = {"inputs": inputs, "filters": numUnits[0], "kernel_size": 1,
                      "activation": tf.nn.relu, "use_bias": True}
            outputs = tf.layers.conv1d(**params)

            # 外层
            params = {"inputs": outputs, "filters": numUnits[1], "kernel_size": 1,
                      "activation": None, "use_bias": True}
            
            # 这里用到了一维卷积，实际上卷积核尺寸还是二维的，只是只需要指定高度，宽度和embedding size的尺寸一致
            outputs = tf.layers.conv1d(**params)
            
            # 残差连接
            outputs += inputs

            # 归一化处理
            outputs = self.layerNormalization(outputs)

        return outputs

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = Transformer(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

Tensor("multihead_attention/MatMul_1:0", shape=(?, 180, 25), dtype=float32)
Tensor("multihead_attention/add:0", shape=(?, 180, 200), dtype=float32)
(?, 180, 200)
(200,)
(?, 180, 200)
(200,)
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name multihead_attention/dense/kernel:0/grad/hist is illegal; using multihead_attention/dense/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name multihead_attention/dense/kernel:0/grad/sparsity is illegal; using multihead_attention/dense/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name multihead_attention/dense/bias:0/grad/hist is illegal; using multihead_attention/dense/bias_0/grad/hist instead.
INFO:tensorflow:Summary name multihead_attention/dense/bias:0/grad/sparsity is illegal; using multihead_attention/dense/bias_0/grad/sparsity instead.
IN

2019-01-02T18:17:53.727658, step: 27, loss: 1.3452759981155396, acc: 0.5078, auc: 0.5133, precision: 0.5833, recall: 0.1077
2019-01-02T18:17:53.898459, step: 28, loss: 1.1268305778503418, acc: 0.4922, auc: 0.5565, precision: 0.5294, recall: 0.1364
2019-01-02T18:17:54.070868, step: 29, loss: 0.8720773458480835, acc: 0.5469, auc: 0.5438, precision: 0.5128, recall: 0.339
2019-01-02T18:17:54.237686, step: 30, loss: 0.9207226037979126, acc: 0.5312, auc: 0.6112, precision: 0.4944, recall: 0.7458
2019-01-02T18:17:54.403764, step: 31, loss: 1.0523834228515625, acc: 0.5547, auc: 0.581, precision: 0.5408, recall: 0.8154
2019-01-02T18:17:54.569911, step: 32, loss: 0.8471137881278992, acc: 0.5781, auc: 0.6303, precision: 0.593, recall: 0.7286
2019-01-02T18:17:54.751571, step: 33, loss: 0.9071395397186279, acc: 0.5547, auc: 0.5474, precision: 0.6176, recall: 0.5753
2019-01-02T18:17:54.924164, step: 34, loss: 0.7343171238899231, acc: 0.6328, auc: 0.6574, precision: 0.6222, recall: 0.4828
2019-01-02T

2019-01-02T18:18:05.303241, step: 95, loss: 0.550586998462677, acc: 0.7344, auc: 0.8386, precision: 0.75, recall: 0.7031
2019-01-02T18:18:05.473283, step: 96, loss: 0.4618009924888611, acc: 0.8047, auc: 0.8718, precision: 0.8148, recall: 0.7458
2019-01-02T18:18:05.647163, step: 97, loss: 0.25469762086868286, acc: 0.8828, auc: 0.9643, precision: 0.9259, recall: 0.8197
2019-01-02T18:18:05.812290, step: 98, loss: 0.5060341954231262, acc: 0.7578, auc: 0.8921, precision: 0.8864, recall: 0.6
2019-01-02T18:18:05.965936, step: 99, loss: 0.4369860589504242, acc: 0.7969, auc: 0.8909, precision: 0.8723, recall: 0.6721
2019-01-02T18:18:06.134627, step: 100, loss: 0.5808737277984619, acc: 0.7734, auc: 0.8632, precision: 0.725, recall: 0.8923

Evaluation:
2019-01-02T18:18:12.837281, step: 100, loss: 0.4345000607845111, acc: 0.8233153846153843, auc: 0.8978384615384615, precision: 0.8142820512820511, recall: 0.8437974358974358
2019-01-02T18:18:13.002278, step: 101, loss: 0.4922887086868286, acc: 0.812

2019-01-02T18:18:23.137792, step: 160, loss: 0.25392234325408936, acc: 0.9062, auc: 0.9601, precision: 0.8961, recall: 0.9452
2019-01-02T18:18:23.308920, step: 161, loss: 0.32933589816093445, acc: 0.8672, auc: 0.9836, precision: 0.8, recall: 0.9846
2019-01-02T18:18:23.481893, step: 162, loss: 0.2659507095813751, acc: 0.8984, auc: 0.9633, precision: 0.8814, recall: 0.8966
2019-01-02T18:18:23.656023, step: 163, loss: 0.18990613520145416, acc: 0.9062, auc: 0.9822, precision: 0.9655, recall: 0.8485
2019-01-02T18:18:23.831976, step: 164, loss: 0.31363824009895325, acc: 0.8203, auc: 0.98, precision: 1.0, recall: 0.6515
2019-01-02T18:18:24.008460, step: 165, loss: 0.3109046518802643, acc: 0.8672, auc: 0.9482, precision: 0.9412, recall: 0.7742
2019-01-02T18:18:24.174742, step: 166, loss: 0.21396152675151825, acc: 0.8906, auc: 0.9717, precision: 0.9189, recall: 0.8947
2019-01-02T18:18:24.329520, step: 167, loss: 0.42832863330841064, acc: 0.8672, auc: 0.9629, precision: 0.8, recall: 0.9677
2019-

2019-01-02T18:18:41.078412, step: 226, loss: 0.2242620438337326, acc: 0.9531, auc: 0.9768, precision: 0.9211, recall: 1.0
2019-01-02T18:18:41.248977, step: 227, loss: 0.18481998145580292, acc: 0.9375, auc: 0.9816, precision: 0.9221, recall: 0.9726
2019-01-02T18:18:41.417226, step: 228, loss: 0.2726507782936096, acc: 0.8906, auc: 0.9556, precision: 0.9153, recall: 0.8571
2019-01-02T18:18:41.586255, step: 229, loss: 0.2042814940214157, acc: 0.875, auc: 0.9808, precision: 0.9672, recall: 0.8082
2019-01-02T18:18:41.750280, step: 230, loss: 0.2316577434539795, acc: 0.9062, auc: 0.9692, precision: 0.9815, recall: 0.8281
2019-01-02T18:18:41.918606, step: 231, loss: 0.191634863615036, acc: 0.9141, auc: 0.9788, precision: 0.9821, recall: 0.8462
2019-01-02T18:18:42.095287, step: 232, loss: 0.20064033567905426, acc: 0.8984, auc: 0.9773, precision: 0.8676, recall: 0.9365
2019-01-02T18:18:42.265060, step: 233, loss: 0.24074290692806244, acc: 0.9297, auc: 0.9698, precision: 0.9286, recall: 0.942
201

2019-01-02T18:18:52.359568, step: 292, loss: 0.21591730415821075, acc: 0.9219, auc: 0.9734, precision: 0.9206, recall: 0.9206
2019-01-02T18:18:52.527000, step: 293, loss: 0.1911626160144806, acc: 0.9375, auc: 0.9792, precision: 0.9643, recall: 0.9
2019-01-02T18:18:52.697579, step: 294, loss: 0.19775879383087158, acc: 0.9141, auc: 0.9763, precision: 0.9423, recall: 0.8596
2019-01-02T18:18:52.868217, step: 295, loss: 0.25683534145355225, acc: 0.8906, auc: 0.959, precision: 0.9412, recall: 0.8136
2019-01-02T18:18:53.041812, step: 296, loss: 0.23172816634178162, acc: 0.9141, auc: 0.9692, precision: 0.95, recall: 0.8769
2019-01-02T18:18:53.211920, step: 297, loss: 0.21315714716911316, acc: 0.9141, auc: 0.9733, precision: 0.9516, recall: 0.8806
2019-01-02T18:18:53.388017, step: 298, loss: 0.1779269129037857, acc: 0.9375, auc: 0.9845, precision: 0.918, recall: 0.9492
2019-01-02T18:18:53.557974, step: 299, loss: 0.15868377685546875, acc: 0.9375, auc: 0.985, precision: 0.9412, recall: 0.9412
20

2019-01-02T18:19:10.427304, step: 358, loss: 0.07232265174388885, acc: 0.9766, auc: 0.9995, precision: 0.9483, recall: 1.0
2019-01-02T18:19:10.602258, step: 359, loss: 0.14428497850894928, acc: 0.9688, auc: 0.9832, precision: 0.9692, recall: 0.9692
2019-01-02T18:19:10.773311, step: 360, loss: 0.05570978671312332, acc: 0.9844, auc: 0.999, precision: 1.0, recall: 0.9615
2019-01-02T18:19:10.946128, step: 361, loss: 0.04782415181398392, acc: 0.9922, auc: 0.9988, precision: 1.0, recall: 0.9821
2019-01-02T18:19:11.118112, step: 362, loss: 0.06263410300016403, acc: 0.9844, auc: 0.9995, precision: 1.0, recall: 0.974
2019-01-02T18:19:11.290078, step: 363, loss: 0.03565814718604088, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:11.467753, step: 364, loss: 0.036555152386426926, acc: 0.9922, auc: 1.0, precision: 0.9863, recall: 1.0
2019-01-02T18:19:11.638296, step: 365, loss: 0.05434255674481392, acc: 0.9922, auc: 0.9995, precision: 0.9848, recall: 1.0
2019-01-02T18:19:11.808033

2019-01-02T18:19:28.426677, step: 424, loss: 0.045727893710136414, acc: 0.9922, auc: 0.9988, precision: 0.9865, recall: 1.0
2019-01-02T18:19:28.598747, step: 425, loss: 0.045645684003829956, acc: 0.9922, auc: 1.0, precision: 0.9825, recall: 1.0
2019-01-02T18:19:28.773276, step: 426, loss: 0.09307089447975159, acc: 0.9609, auc: 0.9956, precision: 0.9821, recall: 0.9322
2019-01-02T18:19:28.940930, step: 427, loss: 0.050742968916893005, acc: 0.9688, auc: 0.9998, precision: 1.0, recall: 0.9286
2019-01-02T18:19:29.107294, step: 428, loss: 0.05154186859726906, acc: 0.9766, auc: 0.9995, precision: 1.0, recall: 0.95
2019-01-02T18:19:29.275032, step: 429, loss: 0.03667081519961357, acc: 0.9922, auc: 0.9998, precision: 1.0, recall: 0.9831
2019-01-02T18:19:29.447296, step: 430, loss: 0.048541005700826645, acc: 0.9844, auc: 0.9995, precision: 0.9853, recall: 0.9853
2019-01-02T18:19:29.615127, step: 431, loss: 0.07922790199518204, acc: 0.9766, auc: 0.9948, precision: 1.0, recall: 0.9577
2019-01-02T

2019-01-02T18:19:40.242977, step: 492, loss: 0.008098553866147995, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:40.419691, step: 493, loss: 0.02375393733382225, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:40.589147, step: 494, loss: 0.01891218312084675, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:40.762909, step: 495, loss: 0.01117563433945179, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:40.936148, step: 496, loss: 0.010379849001765251, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:41.104146, step: 497, loss: 0.011747412383556366, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:41.269271, step: 498, loss: 0.015958203002810478, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:41.442515, step: 499, loss: 0.011247679591178894, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-02T18:19:41.618813, step: 500, loss: 0.014395460486412048, acc: 0.9922, auc